In [1]:
import pandas as pd
import os
import datetime as dt
import zipfile
import tempfile # for temporary file storage to avoid storing large datasets on github

In [2]:
base_dir = os.path.dirname(os.path.abspath("..\data"))
path = os.path.join(base_dir, r"data")
os.chdir(path)

In [3]:
# os.listdir()

#### Functions

In [4]:
def zip_to_df(file_name):
    # create temporary directory  
    with tempfile.TemporaryDirectory() as tmpdirname:
        # collect data from zip file 
        with zipfile.ZipFile(file_name+".zip") as zip_file:
            file_list = zip_file.namelist()
            for file in file_list:
               zip_file.extract(file, tmpdirname)
            file_path = tmpdirname+"\\"+file_name
            # create and return dataframe
            df = pd.read_csv(file_path)
            return df

#### Total Demand

In [5]:
file_name = "totaldemand_nsw.csv"
totalDemand = zip_to_df(file_name)
totalDemand.head()

,DATETIME,REGIONID,TOTALDEMAND
0,2010-01-01 00:00:00,NSW1,7997.41
1,2010-01-01 00:05:00,NSW1,7946.51
2,2010-01-01 00:10:00,NSW1,7921.55
3,2010-01-01 00:15:00,NSW1,7837.84
4,2010-01-01 00:20:00,NSW1,7781.11


In [6]:
totalDemand['DATETIME'] = pd.to_datetime(totalDemand['DATETIME'])

#### Temperature

In [12]:
file_name = "temperature_nsw.csv"
temperature = zip_to_df(file_name)
temperature.head()

,DATETIME,LOCATION,TEMPERATURE
0,2010-01-01 00:00:00,94766.0,23.1
1,2010-01-01 00:01:00,94766.0,23.1
2,2010-01-01 00:30:00,94766.0,22.9
3,2010-01-01 00:50:00,94766.0,22.7
4,2010-01-01 01:00:00,94766.0,22.6


In [13]:
temperature['DATETIME'] = pd.to_datetime(temperature['DATETIME'])

#### Forecast Demand

In [19]:
file_name = "forecastdemand_nsw.csv"
forecastDemand = zip_to_df(file_name)
forecastDemand.head()

,DATETIME,REGIONID,FORECASTDEMAND,PREDISPATCHSEQNO,PERIODID,LASTCHANGED
0,2010-01-01 00:00:00,NSW1,7832.04,2009123018,71,2009-12-30 12:31:49
1,2010-01-01 00:00:00,NSW1,7832.04,2009123019,70,2009-12-30 13:01:43
2,2010-01-01 00:00:00,NSW1,7832.03,2009123020,69,2009-12-30 13:31:36
3,2010-01-01 00:00:00,NSW1,7832.03,2009123021,68,2009-12-30 14:01:44
4,2010-01-01 00:00:00,NSW1,7830.96,2009123022,67,2009-12-30 14:31:35


In [20]:
forecastDemand['DATETIME'] = pd.to_datetime(forecastDemand['DATETIME'])
forecastDemand['LASTCHANGED'] = pd.to_datetime(forecastDemand['LASTCHANGED'])

#### Energy Prices and Demand

In [40]:
wd = path + r"\Price and Demand - 2010-22"
files = os.listdir(wd)

prices = pd.DataFrame([])
file_count = 0
for file in files:
    if file.endswith(".csv"):
        file_df = pd.read_csv(wd + "\\" + file)
        prices = prices.append(file_df, ignore_index=True)
        file_count += 1
print("Merged", file_count, "files")

prices.head()

Merged 153 files


,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,NSW1,2010/01/01 00:30:00,7809.31,21.5,TRADE
1,NSW1,2010/01/01 01:00:00,7483.69,20.98,TRADE
2,NSW1,2010/01/01 01:30:00,7117.23,20.06,TRADE
3,NSW1,2010/01/01 02:00:00,6812.03,17.93,TRADE
4,NSW1,2010/01/01 02:30:00,6544.33,17.43,TRADE


In [30]:
prices['SETTLEMENTDATE'] = pd.to_datetime(prices['SETTLEMENTDATE'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

#### Calculated Columns

In [36]:
totalDemand["DATE"] = totalDemand["DATETIME"].dt.date
forecastDemand["DATE"] = forecastDemand["DATETIME"].dt.date
temperature["DATE"] = temperature["DATETIME"].dt.date
prices["DATE"] = prices["SETTLEMENTDATE"].dt.date

#### Remove redundant columns

In [37]:
totalDemand = totalDemand.drop(columns=["REGIONID"])
forecastDemand = forecastDemand.drop(columns=["REGIONID"])
temperature = temperature.drop(columns=["LOCATION"])
prices = prices.drop(columns=["REGION","PERIODTYPE"])

#### Merge Datasets

Need to look at aggregating datasets on DATE
* totalDemand: aggregate every X number of hours (DATETIME), each aggregate as a column (e.g. 00-06, 06-12, 12-18, 18-00)
* forecastDemand: aggregate every X number of hours (LAST CHANGED), each aggregate as a column (see above)
* temperature: aggregate every X number of hours (DATETIME), each aggregate as a column (see above)
* prices: aggregate every X number of hours (SETTLEMENTDATE), each aggregate as a column (see above)

In [38]:
# df = forecastDemand.merge(temperature, on='DATE')
# df = df.merge(totalDemand, on='DATE')
# df = df.merge(prices, on="DATE")
# df 